## Event Classification from EEG

Data can be found here: https://ucmerced.box.com/s/m8tpshib6zztio9q34h9fh9ikhhl4a3m

Nam Le Dong

### Introduction


The following analyes EEG data taken in experiments where participants where exposed to light and sound events. This code cleans that 32 channel EEG data and uses a Convolutional Neural network (CNN) model to classify the time following light or sound events by which event occured.

### Data Prep



In [1]:
# setting the random seed for reproducibility
import random
seed=42
random.seed(seed)

In [2]:
# import libraries 
import itertools
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

### Load the Data

In [3]:
# eeg1 and events1 are the test data from a single person
# code assumes eeg1 and events1 are csv files in the current working directory

eeg1 = pd.read_csv("eeg1.csv", delimiter="\t")
new_columns = eeg1.columns.values 
new_columns[0] = 'time'     
new_columns[33] = 'sample' 
eeg1.columns = new_columns

events1 = pd.read_csv("events1.csv") #, delimiter="\t"

In [5]:
eeg1.head()

,time,FP1,FPZ,FP2,F7,F3,FZ,F4,F8,FC5,...,P7,P3,PZ,P4,P8,POZ,O1,OZ,O2,sample
0,0.0000,-11.5744,-6.2119,-0.5371,-7.6891,-6.4002,2.1904,6.8447,7.4843,3.7198,...,3.4748,6.0962,9.7316,6.9366,2.3333,6.3835,0.5040,3.3984,2.0209,NaN
1,0.9766,-15.1140,-7.7239,-1.9460,-7.8315,-4.6182,1.0912,5.3037,5.1452,0.5942,...,6.7514,7.2011,10.3467,7.1375,2.0744,7.3299,1.4737,4.6642,3.4500,NaN
2,1.9531,-18.3309,-9.1054,-3.4390,-7.5311,-2.7676,-0.1498,3.3870,2.3100,-2.3004,...,9.6490,8.2395,10.7950,7.1424,1.7050,8.1615,2.2646,5.8274,4.7750,NaN
3,2.9297,-21.0045,-10.2607,-4.9292,-6.7343,-0.9010,-1.4752,1.1906,-0.9090,-4.8420,...,12.1156,9.1695,11.0409,6.9196,1.1848,8.8291,2.9057,6.8464,5.8887,NaN
4,3.9062,-22.9476,-11.1097,-6.3279,-5.4243,0.9241,-2.8213,-1.1634,-4.3681,-6.9461,...,14.1362,9.9566,11.0647,6.4538,0.4847,9.2956,3.4476,7.6932,6.6995,NaN


In [6]:
events1.head()

,number,latency,type,duration
0,1,1,0,0
1,2,66164,6,0
2,3,66760,6,0
3,4,67374,6,0
4,5,67989,6,0


In [8]:
events1.shape

(4283, 4)

In [9]:
eeg1.shape

(3469303, 34)

In [10]:
# subsample of the data to ease building the model, unused in final run
eeg1_smol = eeg1[0:785000]
events1_smol = events1[0:1000]

In [11]:
eeg1_smol.shape

(785000, 34)

In [12]:
events1_smol.shape

(1000, 4)

In [13]:
# Toy data generator, unused in final run

def generate_eeg(samples, time_steps, n_features, event_types):
    # samples is Int number of trials 
    # time_steps is Int length of each trial in ms
    # n_features is Int number of EEG channels
    # event_types is Int number of stimula like lights and flashes
    signals = generate_signals(samples, time_steps, n_features)
    events = generate_events(event_types, samples)
    events_1hot = one_hot_events(events)
    return signals, events_1hot

# helper function (generate_eeg) for making EEG signal data
def generate_signals(samples, time_steps, n_features):
    # data types same as main function
    signals = np.random.random((samples, time_steps, n_features))
    return signals

# helper function (generate_eeg) for making one sample per event an
def generate_events(event_types, samples):
    # data types same as main function
    events = np.random.randint(1, event_types, samples)
    return events

In [14]:
# takes in eeg dataframe and event dataframe, cleans them, 1hot encodes the events
def clean_eeg(eeg, events, event_interval_length, eeg_slice_length):
    #event_list = []
    array_list = [] 
    index_list = []
    eeg = standardize_eeg(eeg) # function for standardizing the eeg readings
    #events_new = build_zero_events(events)
    # iterate over the rows of the events and slice out the corresponding eeg data
    for index, row in itertools.islice(events.iterrows(), event_interval_length): # loop through events data
        #build_event_list(row, event_list) #
        tmin, tmax = build_event_intervals(row, events)
        eeg_slice = cut_event_intervals(eeg, tmin, tmax)
        array_list, index_list = build_array(eeg_slice, eeg_slice_length, 
                                             index, index_list, array_list)
    y_int = events.iloc[index_list] # take the event types for the correct index
    y_int = y_int['type'].values    # take just the event types as an array
    #y_int = y_int.as_matrix()            # save the event types as a matrix
    #y, lb = one_hot_events(y_int)        # one-hot the event types and save the binarizer
    X = np.stack(array_list, axis = 0)   # stack the arrays so the whole thing is 3D
    return X, y_int                     # return the data, outputs, and the binarizer
    
        
def build_event_list(row, event_list):
    # helper function to pull event types out of event data in the right order
    event_type = getattr(row, "type")
    event_list.append(event_type)
        
def build_event_intervals(row, events):
    # helper function to get the time intervals associated with each event
    tmin = getattr(row, "latency")
    tmin_in = getattr(row, "number")
    tmax_in = tmin_in + 1
    tmax = events1.loc[tmax_in, "latency"]
    return tmin, tmax

def cut_event_intervals(eeg, tmin, tmax):
    # helper function to slice up the eeg data so each slice is associated with one event
    eeg_slice = eeg.loc[(eeg["time"] > tmin) & (eeg["time"] < tmax)]
    eeg_slice.drop(["time", "sample"], axis = 1, inplace = True)
    return eeg_slice
    
def build_array(eeg_slice, eeg_slice_length, index, index_list, array_list):
    # helper function to build an array out of the eeg slices and pad them out to a standard length
    if len(eeg_slice) < eeg_slice_length:
        index_list.append(index)
        eeg_matrix = eeg_slice.as_matrix()
        padded_matrix = np.pad(eeg_matrix, ((0, eeg_slice_length - len(eeg_matrix)), (0,0)),
                                   'constant', constant_values=0)
        array_list.append(padded_matrix)
    return array_list, index_list

def one_hot_events(events):
    # helper function for one-hot encoding the events
    events_list = list(events)
    lb = preprocessing.LabelBinarizer()
    lb.fit(events_list)
    events_1hot = lb.transform(events_list)
    return events_1hot, lb

def invert_one_hot(events, lb):
    # function for decoding one-hot, binarizer made in one_hot_events
    inv_events = lb.inverse_transform(events)
    return inv_events

In [15]:
def standardize_eeg(eeg_data):
    # breaks apart an eeg dataframe, scales the eeg readings, and reassmbles it into a dataframe
    column_list = eeg_data.columns[1:33]
    time = eeg_data['time']
    sample = eeg_data['sample']
    eeg_array = eeg_data[column_list]
    eeg_stnd = scale_data(eeg_array)
    eeg_stnd_df = pd.DataFrame(eeg_stnd, index=eeg_data.index, columns=column_list)
    eeg_stnd = pd.concat([time, eeg_stnd_df, sample], axis =1)
    return eeg_stnd

def scale_data(unscaled_data):
    # helper function for standardize_eeg, fits a scaler and transforms the data 
    scaler = StandardScaler()
    scaler.fit(unscaled_data)
    scaled_data = scaler.transform(unscaled_data)
    return scaled_data

In [16]:
# This is unused code for breaking up the "nothing happened" periods of the eeg data 
# to associate with "type 0" events. 

import math
time_steps = 1300

def build_zero_events(event_data, time_steps=time_steps):
    new_events = build_new_events(event_data, time_steps)
    events = zero_events(event_data, new_events)
    return events


def build_new_events(event_data, time_steps= time_steps):
    first_event_time = event_data['latency'].loc[1]
    number_new_intervals = math.floor(first_event_time / time_steps)
    df = pd.DataFrame(columns=['number', 'latency', 'type', 'duration'],index = range(number_new_intervals) )
    latency = 0
    for t in range(number_new_intervals):
        latency += 1300
        df.loc[t].latency = latency
        df.loc[t].type = 0
    return df

def zero_events(event_data, new_events):
    events_zeros = event_data[event_data.latency != 1]
    events_zeros= new_events.append(events_zeros)
    events_zeros = events_zeros.reset_index(drop=True)
    events_zeros['number'] = events_zeros.index + 1
    return events_zeros

### Model



In [17]:
# full dataset parameters

# define model parameters
samples = 3625  # how many trials of eeg data
n_features = 32  # how many channels of eeg in each sample
time_steps = 1300 # how many ms was each sample run for
event_types = 2 #len(set(y))  # how many different event types (light, sound, etc) are there # 6 large, 4 smol

In [18]:
# get the data into useable form and store as X and y
X, y = clean_eeg(eeg1, events1, samples, time_steps)  #4250 long, 998 short, 4330 long enhanced

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [19]:
X.shape

(3554, 1300, 32)

In [21]:
X[0]

array([[ 0.22334067,  0.16348891,  0.2673775 , ...,  0.13175228,
         0.25368819, -0.3451777 ],
       [ 0.21936408,  0.13216201,  0.25447713, ...,  0.0046344 ,
         0.20014994, -0.35425198],
       [ 0.21598459,  0.10373928,  0.23813144, ..., -0.14758523,
         0.12719416, -0.36555702],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [22]:
y.shape

(3554,)

In [24]:
# removes the minor event types. There were only a couple hundred examples of each, whereas the used events had a 
# couple thousand examples
remove_list = [0,2,4,5,6]              # designate unwanted event types
drop_list = np.isin(y, remove_list)    # create a list of indices associated with unwanted events                  
drop_array = np.array(drop_list)       # make the list of indices to drop into an array

In [25]:
y

array([6, 6, 6, ..., 1, 1, 1], dtype=int64)

In [26]:
# make X, y's with the unwanted events removed
y_short_int = y[np.isin(y,remove_list, invert=True)]
X_short = X[np.isin(y, remove_list, invert=True)]

In [27]:
# one hot encode the y data without the unwanted events
y_short, lb = one_hot_events(y_short_int) 

In [28]:
X_short.shape

(3056, 1300, 32)

In [29]:
y_short.shape

(3056, 1)

In [30]:
y_short

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [31]:
from sklearn.model_selection import StratifiedShuffleSplit

# use strat. shuffle split to get indices for test and training data 
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=seed)
sss.get_n_splits(X_short, y_short)

2

In [32]:
# take the indices generated by stratified shuffle split and make the test and training datasets
for train_index, test_index in sss.split(X_short, y_short):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_short[train_index], X_short[test_index]
    y_train, y_test = y_short[train_index], y_short[test_index]

In [33]:
X_train.shape

(2444, 1300, 32)

In [34]:
X_test.shape

(612, 1300, 32)

In [37]:
y_train.shape

(2444, 1)

In [38]:
y_test.shape

(612, 1)

In [ ]:
y_train[0:5]

In [ ]:
y_test[0:5]

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,BatchNormalization, Embedding, Flatten, Conv1D, Activation, MaxPooling1D

# code for building an LSTM with 100 neurons and dropout. Runs for 50 epochs

model = Sequential()
# CNN 1

model.add(Conv1D(filters=32, kernel_size=3, 
          input_shape=(time_steps, n_features)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))    
model.add(Dropout(0.2))
    
# CNN 2
model.add(Conv1D(filters=64,  kernel_size=3, padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))    
model.add(Dropout(0.2))
        
# CNN 3
model.add(Conv1D(filters=128,  kernel_size=3, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))    
model.add(Dropout(0.2))

# Output
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=256, epochs=50)
score = model.evaluate(X_test, y_test, batch_size=256)

Epoch 1/50
2444/2444 [==============================] - 3s 1ms/step - loss: 1.9653 - acc: 0.4861
Epoch 2/50
2444/2444 [==============================] - 1s 505us/step - loss: 0.7792 - acc: 0.5327
Epoch 3/50
2444/2444 [==============================] - 1s 499us/step - loss: 0.6991 - acc: 0.5790
Epoch 4/50
2444/2444 [==============================] - 1s 499us/step - loss: 0.6677 - acc: 0.6072
Epoch 5/50
2444/2444 [==============================] - 1s 505us/step - loss: 0.6497 - acc: 0.6322
Epoch 6/50
2444/2444 [==============================] - 1s 499us/step - loss: 0.6417 - acc: 0.6244
Epoch 7/50
2444/2444 [==============================] - 1s 518us/step - loss: 0.6212 - acc: 0.6698
Epoch 8/50
2444/2444 [==============================] - 1s 505us/step - loss: 0.6255 - acc: 0.6624
Epoch 9/50
2444/2444 [==============================] - 1s 505us/step - loss: 0.5656 - acc: 0.7324
Epoch 10/50
2444/2444 [==============================] - 1s 511us/step - loss: 0.4847 - acc: 0.8011
Epoch 11/50

In [46]:
score

[0.18964731654310538, 0.9689542530408872]

In [47]:
print("Accuracy: %.2f%%" % (score[1]*100))

Accuracy: 96.90%
